In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audios/audios_waves/sea waves/2-77945-B.wav
/kaggle/input/audios/audios_waves/sea waves/1-91359-A.wav
/kaggle/input/audios/audios_waves/sea waves/5-219379-C.wav
/kaggle/input/audios/audios_waves/sea waves/3-118658-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-182613-B.wav
/kaggle/input/audios/audios_waves/sea waves/3-164120-A.wav
/kaggle/input/audios/audios_waves/sea waves/2-50668-A.wav
/kaggle/input/audios/audios_waves/sea waves/1-28135-A.wav
/kaggle/input/audios/audios_waves/sea waves/5-216370-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-169127-B.wav
/kaggle/input/audios/audios_waves/sea waves/1-47250-B.wav
/kaggle/input/audios/audios_waves/sea waves/4-157611-B.wav
/kaggle/input/audios/audios_waves/sea waves/1-61252-A.wav
/kaggle/input/audios/audios_waves/sea waves/3-155642-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-165823-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-157611-A.wav
/kaggle/input/audios/audios_waves/sea waves/1-39901-A.wav
/kag

In [55]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
import librosa
import librosa
import soundfile as sf

In [120]:
# Now, X contains the extracted MFCC features, and y contains corresponding labels.
def process_audio_folder(folder_path):
    features = []
    labels = []

    for label, animal_folder in enumerate(os.listdir(folder_path)):
        for audio_file in os.listdir(os.path.join(folder_path, animal_folder)):
            audio_path = os.path.join(folder_path, animal_folder, audio_file)

            # Extract MFCC features
#             mfccs, chroma, mel, contrast, tonnetz = extract_features(audio_path)
#             mfccs, mel = extract_features(audio_path)
#             mfccs, contrast = extract_features(audio_path)
            #mfccs, tonnetz = extract_features(audio_path)
            mfccs,chroma, tonnetz = extract_features(audio_path)
            if mfccs is not None:
                # Concatenate all the features into a single array
#                 all_features = np.concatenate([mfccs, chroma, mel, contrast, tonnetz])
#                 all_features = np.concatenate([mfccs, chroma])
#                 all_features = np.concatenate([mfccs, mel])
                #all_features = np.concatenate([mfccs, contrast])
#                 all_features = np.concatenate([mfccs, tonnetz])
                all_features = np.concatenate([chroma, tonnetz])
                features.append(all_features)
                labels.append(label)

    return np.array(features), np.array(labels)

In [124]:
def extract_features(file_name=None):
    if file_name:
        print('Extracting', file_name)
        X, sample_rate = sf.read(file_name, dtype='float32')
    if X.ndim > 1:
        X = X[:, 0]
    X = X.T

    # short term fourier transform
    stft = np.abs(librosa.stft(X))

    # mfcc (mel-frequency cepstrum) 40
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=192).T, axis=0)

    # chroma
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)

    # melspectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)

    # spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)     
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)

#     return mfccs, chroma, mel, contrast, tonnetz
#     return  chroma, mfccs
#     return  mfccs, mel
#     return mfccs,contrast
#     return mfccs,tonnetz
    return mfccs,chroma,tonnetz

In [ ]:

# Example usage:
folder_path = "/kaggle/input/audios/audios_waves/"
X, y = process_audio_folder(folder_path)

Extracting /kaggle/input/audios/audios_waves/sea waves/2-77945-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-91359-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/5-219379-C.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/3-118658-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/4-182613-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/3-164120-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/2-50668-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-28135-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/5-216370-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/4-169127-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-47250-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/4-157611-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-61252-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/3-155642-A.wav
Extracting /kaggle/input/a

In [97]:
print(X)
print(y)

[[ -40.04023743   67.37039948  -40.07915878 ...   15.52179051
    15.60453692   38.15801709]
 [-112.22832489  144.25550842  -40.78744125 ...   14.79348627
    14.64364974   46.7794436 ]
 [-188.3168335   180.23867798  -98.50474548 ...   15.07703704
    15.58654811   42.40457381]
 ...
 [-712.05065918   37.24608612  -12.9059515  ...   12.55520658
    12.56692302   17.21573922]
 [-509.793396     37.5605278    -9.3521595  ...   18.2388661
    16.75873944   23.81937534]
 [-494.49221802   51.83587265   -8.53918266 ...   16.12255932
    16.39123257   28.06008409]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [101]:

# Step 1: Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)



In [102]:
# Step 2: Data Normalization
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)



In [103]:
# Step 3: Building the Neural Network
model = Sequential()
model.add(Dense(128, input_dim=X_train_normalized.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))



In [105]:
# Step 4: Model Compilation
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [106]:

# Step 5: Model Training
model.fit(X_train_normalized, y_train, epochs=40, batch_size=32, validation_split=0.2)



Epoch 1/40
7/7 [==============================] - 1s 25ms/step - loss: 2.1834 - accuracy: 0.1471 - val_loss: 2.2039 - val_accuracy: 0.1154
Epoch 2/40
7/7 [==============================] - 0s 6ms/step - loss: 2.1111 - accuracy: 0.1863 - val_loss: 2.1835 - val_accuracy: 0.1346
Epoch 3/40
7/7 [==============================] - 0s 6ms/step - loss: 2.0646 - accuracy: 0.2304 - val_loss: 2.1350 - val_accuracy: 0.2115
Epoch 4/40
7/7 [==============================] - 0s 5ms/step - loss: 2.0158 - accuracy: 0.2696 - val_loss: 2.0952 - val_accuracy: 0.1923
Epoch 5/40
7/7 [==============================] - 0s 5ms/step - loss: 1.9633 - accuracy: 0.3235 - val_loss: 2.0726 - val_accuracy: 0.1538
Epoch 6/40
7/7 [==============================] - 0s 5ms/step - loss: 1.9083 - accuracy: 0.3676 - val_loss: 2.0333 - val_accuracy: 0.1923
Epoch 7/40
7/7 [==============================] - 0s 5ms/step - loss: 1.8486 - accuracy: 0.3775 - val_loss: 1.9781 - val_accuracy: 0.1923
Epoch 8/40
7/7 [=================

In [107]:
# Step 6: Model Evaluation
accuracy = model.evaluate(X_test_normalized, y_test)[1]
print(f'Test Accuracy: {accuracy}')


6/6 [==============================] - 0s 2ms/step - loss: 0.9663 - accuracy: 0.6395
Test Accuracy: 0.6395348906517029


Test Accuracy: 0.6976743936538696  all features  mfccs, chroma, mel, contrast, tonnetz  test_size=0.4  epochs=40

Test Accuracy: 0.6976743936538696   features  mfccs, chroma  test_size=0.4  epochs=40

Test Accuracy: 0.6627907156944275    features  mfccs, mel  test_size=0.4  epochs=40

Test Accuracy: 0.6627907156944275     features  mfccs, contrast  test_size=0.4  epochs=40


Test Accuracy: 0.6511628031730652 features  mfccs, contrast  test_size=0.4  epochs=192



Test Accuracy: 0.6511628031730652 features  mfccs, contrast  test_size=0.4  epochs=40 batch_size=80



Test Accuracy:  0.6395348906517029 features  mfccs, tonnetz  test_size=0.4  epochs=40 batch_size=32
